In [2]:
## THIS SCRIPT IS USED TO COMPARE SPARK AND DUCKDB PERFORMANCE ON A SINGLE NODE
## IT USES A SINGLE NODE SPARK CLUSTER AND A SINGLE NODE DUCKDB 


import pandas as pd
import numpy as np
import time
from pyspark.sql import SparkSession    
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [3]:
#this is a small file , DATASET 1

df_pandas = pd.read_json("D:\Project\Spark_project\combined_file.txt",lines=True)


<>:3: SyntaxWarning: invalid escape sequence '\P'
<>:3: SyntaxWarning: invalid escape sequence '\P'
C:\Users\raksh\AppData\Local\Temp\ipykernel_17808\3178942341.py:3: SyntaxWarning: invalid escape sequence '\P'
  df_pandas = pd.read_json("D:\Project\Spark_project\combined_file.txt",lines=True)


In [4]:
df_pandas.head()

,type,dt,payload
0,new_adsb,2025-01-31 00:00:00.330843,"{'hex': 'ab35d3', 'flight': 'DAL136 ', 'alt_b..."
1,new_adsb,2025-01-31 00:00:00.336607,"{'hex': 'c03f37', 'flight': 'WJA598 ', 'alt_b..."
2,new_adsb,2025-01-31 00:00:00.336872,"{'hex': 'c06a75', 'flight': 'TSC359 ', 'alt_b..."
3,new_adsb,2025-01-31 00:00:00.337039,"{'hex': 'c078ba', 'flight': 'TSC939 ', 'alt_b..."
4,new_adsb,2025-01-31 00:00:00.337196,"{'hex': 'c027da', 'flight': 'CJT990 ', 'alt_b..."


In [5]:
pandas_df = pd.json_normalize(df_pandas['payload'])
pandas_df = df_pandas[['dt']].join(pandas_df)
pandas_df.head()

,dt,hex,flight,alt_baro,alt_geom,gs,track,baro_rate,squawk,emergency,...,nav_modes,geom_rate,nav_altitude_fms,tas,mag_heading,true_heading,ias,mach,track_rate,roll
0,2025-01-31 00:00:00.330843,ab35d3,DAL136,37000,36475.0,552.7,41.3,0.0,5740,none,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-01-31 00:00:00.336607,c03f37,WJA598,23925,23325.0,352.3,22.7,-960.0,0512,none,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-01-31 00:00:00.336872,c06a75,TSC359,24725,24125.0,482.7,47.5,-1856.0,1541,none,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-01-31 00:00:00.337039,c078ba,TSC939,15175,14600.0,420.6,48.1,-2944.0,6107,none,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-31 00:00:00.337196,c027da,CJT990,30000,29450.0,343.8,229.1,0.0,0630,none,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Dataset 2 

df_flight_hex = pd.read_json("basic-ac-db.json",lines=True)
df_flight_hex.head()

,icao,reg,icaotype,year,manufacturer,model,ownop,faa_pia,faa_ladd,short_type,mil
0,a05fc7,N123KQ,None,None,None,None,VALIAIR LIMITED COMPANY AS TRUSTEE,False,False,None,False
1,a133a3,N177ES,None,None,None,None,STEVE WESSELS,False,False,None,False
2,a76176,N575EX,None,None,None,None,SPIRO BUAS,False,False,None,False
3,a0139a,N104AK,None,None,None,None,PACIFIC DIVERSIFIED INVESTMENTS INC,False,False,None,False
4,a2bc89,N2758K,None,None,None,None,CANCELLED/NOT ASSIGNED,False,False,None,False


In [5]:
df_flight_hex = df_flight_hex.rename(columns={'icao': 'hex'})
df_flight_hex.head()

,hex,reg,icaotype,year,manufacturer,model,ownop,faa_pia,faa_ladd,short_type,mil
0,a05fc7,N123KQ,None,None,None,None,VALIAIR LIMITED COMPANY AS TRUSTEE,False,False,None,False
1,a133a3,N177ES,None,None,None,None,STEVE WESSELS,False,False,None,False
2,a76176,N575EX,None,None,None,None,SPIRO BUAS,False,False,None,False
3,a0139a,N104AK,None,None,None,None,PACIFIC DIVERSIFIED INVESTMENTS INC,False,False,None,False
4,a2bc89,N2758K,None,None,None,None,CANCELLED/NOT ASSIGNED,False,False,None,False


In [11]:
print(pandas_df.shape)
print(df_flight_hex.shape)

(1829647, 42)
(603579, 11)


In [13]:
new_df = pd.merge(pandas_df, df_flight_hex, on='hex', how='inner')
new_df.shape

(1829150, 52)

In [14]:
new_df.head()

,dt,hex,flight,alt_baro,alt_geom,gs,track,baro_rate,squawk,emergency,...,reg,icaotype,year,manufacturer,model,ownop,faa_pia,faa_ladd,short_type,mil
0,2025-01-31 00:00:00.330843,ab35d3,DAL136,37000,36475.0,552.7,41.3,0.0,5740,none,...,N821NW,A333,2007,AIRBUS,A330-323,DELTA AIR LINES INC,False,False,L2J,False
1,2025-01-31 00:00:00.336607,c03f37,WJA598,23925,23325.0,352.3,22.7,-960.0,0512,none,...,C-FXYK,B38M,2023,THE BOEING COMPANY,737-8,Westjet,False,False,L2J,False
2,2025-01-31 00:00:00.336872,c06a75,TSC359,24725,24125.0,482.7,47.5,-1856.0,1541,none,...,C-GOIE,A21N,2019,AIRBUS S.A.S,A321-271NX,Air Transat A T Inc.,False,False,L2J,False
3,2025-01-31 00:00:00.337039,c078ba,TSC939,15175,14600.0,420.6,48.1,-2944.0,6107,none,...,C-GTSR,A332,2008,Airbus Industrie,A330-243,Air Transat A T Inc.,False,False,L2J,False
4,2025-01-31 00:00:00.337196,c027da,CJT990,30000,29450.0,343.8,229.1,0.0,0630,none,...,C-FPCJ,B763,1995,THE BOEING COMPANY,767-306,Cargojet Airways Ltd.,False,False,L2J,False


In [16]:
new_df['model'].value_counts().sort_values(ascending=False).head(10)

model
DHC-8-402                   233318
A319-114                    113000
BD-500-1A11                  95945
737-8                        88622
A321-211                     69841
CL-600-2D24                  61014
                             60281
ERJ 190-400                  50697
CL-600-2D24 (Series 900)     47356
ERJ 170-200 SU               40617
Name: count, dtype: int64

In [17]:
spark = SparkSession.builder \
    .appName("LoadExtractedData_2") \
    .master("local[*]") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

In [18]:
df_spark = spark.read.json("D:\Project\Spark_project\combined_file.txt")

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\raksh\AppData\Local\Temp\ipykernel_31192\1731887496.py:1: SyntaxWarning: invalid escape sequence '\P'
  df_spark = spark.read.json("D:\Project\Spark_project\combined_file.txt")


In [19]:
from pyspark.sql.functions import col, to_date

df_spark  = df_spark.select(
    col("type").alias("Type"),
    col("dt").alias("DT"),
    col("payload.hex").alias("HexID"),
    col("payload.flight").alias("FlightNumber"),
    col("payload.alt_baro").alias("AltBaro"),
    col("payload.alt_geom").alias("AltGeom"),
    col("payload.gs").alias("GroundSpeed"),
    col("payload.track").alias("Track"),
    col("payload.baro_rate").alias("BaroRate"),
    col("payload.squawk").alias("Squawk"),
    col("payload.emergency").alias("Emergency"),
    col("payload.category").alias("Category"),
    col("payload.nav_qnh").alias("NavQNH"),
    col("payload.nav_altitude_mcp").alias("NavAltitudeMCP"),
    col("payload.nav_heading").alias("NavHeading"),
    col("payload.lat").alias("Latitude"),
    col("payload.lon").alias("Longitude"),
    col("payload.nic").alias("NIC"),
    col("payload.rc").alias("RC"),
    col("payload.seen_pos").alias("SeenPos"),
    col("payload.version").alias("Version"),
    col("payload.nic_baro").alias("NICBaro"),
    col("payload.nac_p").alias("NAC_P"),
    col("payload.nac_v").alias("NAC_V"),
    col("payload.sil").alias("SIL"),
    col("payload.sil_type").alias("SIL_Type"),
    col("payload.gva").alias("GVA"),
    col("payload.sda").alias("SDA"),
    col("payload.mlat").alias("MLAT"),
    col("payload.tisb").alias("TISB"),
    col("payload.messages").alias("Messages"),
    col("payload.seen").alias("Seen"),
    col("payload.rssi").alias("RSSI")
)





In [20]:
df_spark.show(5)

+--------+--------------------+------+------------+-------+-------+-----------+-----+--------+------+---------+--------+------+--------------+----------+---------+----------+---+---+-------+-------+-------+-----+-----+---+--------+---+---+----+----+--------+----+-----+
|    Type|                  DT| HexID|FlightNumber|AltBaro|AltGeom|GroundSpeed|Track|BaroRate|Squawk|Emergency|Category|NavQNH|NavAltitudeMCP|NavHeading| Latitude| Longitude|NIC| RC|SeenPos|Version|NICBaro|NAC_P|NAC_V|SIL|SIL_Type|GVA|SDA|MLAT|TISB|Messages|Seen| RSSI|
+--------+--------------------+------+------------+-------+-------+-----------+-----+--------+------+---------+--------+------+--------------+----------+---------+----------+---+---+-------+-------+-------+-----+-----+---+--------+---+---+----+----+--------+----+-----+
|new_adsb|2025-01-31 00:00:...|ab35d3|    DAL136  |  37000|  36475|      552.7| 41.3|       0|  5740|     none|      A5|1013.6|         36992|       0.0|44.218048|-75.741316|  8|186|    0.4|

In [28]:
df_spark_flight_hex = spark.read.json("basic-ac-db.json")

In [30]:
df_spark_flight_hex1 = spark.read \
    .option("multiline", "true") \
    .json("basic-ac-db.json")

df_spark_flight_hex.show(5, truncate=False)
df_spark_flight_hex.printSchema()


+--------+-------+------+--------+------------+-----+-----+-----------------------------------+------+----------+----+
|faa_ladd|faa_pia|icao  |icaotype|manufacturer|mil  |model|ownop                              |reg   |short_type|year|
+--------+-------+------+--------+------------+-----+-----+-----------------------------------+------+----------+----+
|false   |false  |a05fc7|NULL    |NULL        |false|NULL |VALIAIR LIMITED COMPANY AS TRUSTEE |N123KQ|NULL      |NULL|
|false   |false  |a133a3|NULL    |NULL        |false|NULL |STEVE WESSELS                      |N177ES|NULL      |NULL|
|false   |false  |a76176|NULL    |NULL        |false|NULL |SPIRO BUAS                         |N575EX|NULL      |NULL|
|false   |false  |a0139a|NULL    |NULL        |false|NULL |PACIFIC DIVERSIFIED INVESTMENTS INC|N104AK|NULL      |NULL|
|false   |false  |a2bc89|NULL    |NULL        |false|NULL |CANCELLED/NOT ASSIGNED             |N2758K|NULL      |NULL|
+--------+-------+------+--------+------------+-

In [29]:
df_spark_flight_hex.show(5)

+--------+-------+------+--------+------------+-----+-----+--------------------+------+----------+----+
|faa_ladd|faa_pia|  icao|icaotype|manufacturer|  mil|model|               ownop|   reg|short_type|year|
+--------+-------+------+--------+------------+-----+-----+--------------------+------+----------+----+
|   false|  false|a05fc7|    NULL|        NULL|false| NULL|VALIAIR LIMITED C...|N123KQ|      NULL|NULL|
|   false|  false|a133a3|    NULL|        NULL|false| NULL|       STEVE WESSELS|N177ES|      NULL|NULL|
|   false|  false|a76176|    NULL|        NULL|false| NULL|          SPIRO BUAS|N575EX|      NULL|NULL|
|   false|  false|a0139a|    NULL|        NULL|false| NULL|PACIFIC DIVERSIFI...|N104AK|      NULL|NULL|
|   false|  false|a2bc89|    NULL|        NULL|false| NULL|CANCELLED/NOT ASS...|N2758K|      NULL|NULL|
+--------+-------+------+--------+------------+-----+-----+--------------------+------+----------+----+
only showing top 5 rows



In [35]:
df_spark_flight_hex = df_spark_flight_hex.withColumnRenamed("hex", "HexID")
df_spark_flight_hex.show(5)

+--------+-------+------+--------+------------+-----+-----+--------------------+------+----------+----+
|faa_ladd|faa_pia| HexID|icaotype|manufacturer|  mil|model|               ownop|   reg|short_type|year|
+--------+-------+------+--------+------------+-----+-----+--------------------+------+----------+----+
|   false|  false|a05fc7|    NULL|        NULL|false| NULL|VALIAIR LIMITED C...|N123KQ|      NULL|NULL|
|   false|  false|a133a3|    NULL|        NULL|false| NULL|       STEVE WESSELS|N177ES|      NULL|NULL|
|   false|  false|a76176|    NULL|        NULL|false| NULL|          SPIRO BUAS|N575EX|      NULL|NULL|
|   false|  false|a0139a|    NULL|        NULL|false| NULL|PACIFIC DIVERSIFI...|N104AK|      NULL|NULL|
|   false|  false|a2bc89|    NULL|        NULL|false| NULL|CANCELLED/NOT ASS...|N2758K|      NULL|NULL|
+--------+-------+------+--------+------------+-----+-----+--------------------+------+----------+----+
only showing top 5 rows



In [36]:
new_df_spark = df_spark.join(df_spark_flight_hex, "HexID", "inner")
new_df_spark.show(5)

+------+--------+--------------------+------------+-------+-------+-----------+-----+--------+------+---------+--------+------+--------------+----------+--------+---------+----+----+-------+-------+-------+-----+-----+----+--------+----+----+----+----+--------+----+-----+--------+-------+--------+------------+-----+------------+-----+------+----------+----+
| HexID|    Type|                  DT|FlightNumber|AltBaro|AltGeom|GroundSpeed|Track|BaroRate|Squawk|Emergency|Category|NavQNH|NavAltitudeMCP|NavHeading|Latitude|Longitude| NIC|  RC|SeenPos|Version|NICBaro|NAC_P|NAC_V| SIL|SIL_Type| GVA| SDA|MLAT|TISB|Messages|Seen| RSSI|faa_ladd|faa_pia|icaotype|manufacturer|  mil|       model|ownop|   reg|short_type|year|
+------+--------+--------------------+------------+-------+-------+-----------+-----+--------+------+---------+--------+------+--------------+----------+--------+---------+----+----+-------+-------+-------+-----+-----+----+--------+----+----+----+----+--------+----+-----+--------

In [37]:
new_df_spark.groupBy("model").count().orderBy("count", ascending=False).show(10)

+--------------------+------+
|               model| count|
+--------------------+------+
|           DHC-8-402|233318|
|            A319-114|113000|
|         BD-500-1A11| 95945|
|               737-8| 88622|
|            A321-211| 69841|
|         CL-600-2D24| 61014|
|                    | 60281|
|         ERJ 190-400| 50697|
|CL-600-2D24 (Seri...| 47356|
|      ERJ 170-200 SU| 40617|
+--------------------+------+
only showing top 10 rows



In [39]:
from pyspark.sql.functions import count, desc

new_df_spark.groupBy("model") \
    .agg(count("*").alias("TotalCount")) \
    .orderBy(desc("TotalCount")) \
    .show(10)


+--------------------+----------+
|               model|TotalCount|
+--------------------+----------+
|           DHC-8-402|    233318|
|            A319-114|    113000|
|         BD-500-1A11|     95945|
|               737-8|     88622|
|            A321-211|     69841|
|         CL-600-2D24|     61014|
|                    |     60281|
|         ERJ 190-400|     50697|
|CL-600-2D24 (Seri...|     47356|
|      ERJ 170-200 SU|     40617|
+--------------------+----------+
only showing top 10 rows



In [38]:
grouped_df = new_df_spark.groupBy("model").count()
result_df = grouped_df.orderBy("count", ascending=False)
result_df.show(10)


+--------------------+------+
|               model| count|
+--------------------+------+
|           DHC-8-402|233318|
|            A319-114|113000|
|         BD-500-1A11| 95945|
|               737-8| 88622|
|            A321-211| 69841|
|         CL-600-2D24| 61014|
|                    | 60281|
|         ERJ 190-400| 50697|
|CL-600-2D24 (Seri...| 47356|
|      ERJ 170-200 SU| 40617|
+--------------------+------+
only showing top 10 rows

